In [22]:
import numpy as np
import cv2
import time
import requests
import threading
from threading import Thread, Event, ThreadError

In [25]:
class Cam():

    def __init__(self, url):
    
        self.stream = requests.get(url, stream=True)
        self.thread_cancelled = False
        self.thread = Thread(target=self.run)
        print("camera initialised")

    
    def start(self):
        self.thread.start()
        print("camera stream started")
    
    def run(self):
        bytes=''
        while not self.thread_cancelled:
            try:
                bytes+=self.stream.raw.read(1024)
                a = bytes.find('\xff\xd8')
                b = bytes.find('\xff\xd9')
                if a!=-1 and b!=-1:
                    jpg = bytes[a:b+2]
                    bytes= bytes[b+2:]
                    img = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8),cv2.IMREAD_COLOR)
                    cv2.imshow('cam',img)
                    if cv2.waitKey(1) ==27:
                        exit(0)
            except ThreadError:
                self.thread_cancelled = True
        
        
    def is_running(self):
        return self.thread.isAlive()
      
    
    def shut_down(self):
        self.thread_cancelled = True
        #block while waiting for thread to terminate
        while self.thread.isAlive():
            time.sleep(1)
        return True

  
    
if __name__ == "__main__":
    url = 'http://ie.trafficland.com/v1.0/700/full?system=trafficland-www&pubtoken=ad465d1741517b42a094f14809ff94c6143a96294a7bcc0512a5c4a6a4bdc7fd&refreshRate=2000&cache=86044'
    cam = Cam(url)
    cam.start()

camera initialised
camera stream started


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-25-d3d14a959307>", line 19, in run
    bytes+=self.stream.raw.read(1024)
TypeError: Can't convert 'bytes' object to str implicitly

